In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Load the data
file = 'CountyAccessDemographics.csv'
df = pd.read_csv(file)
# check data type
df.dtypes 

state_county         object
Pop2010               int64
PctLA               float64
PctWhite            float64
PctBlack            float64
PctHispanic         float64
PctOtherMinority    float64
LA1and10              int64
dtype: object

In [3]:
# Find null values
for column in df.columns :
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column state_county has 0 null values
Column Pop2010 has 0 null values
Column PctLA has 0 null values
Column PctWhite has 0 null values
Column PctBlack has 0 null values
Column PctHispanic has 0 null values
Column PctOtherMinority has 0 null values
Column LA1and10 has 0 null values


In [4]:
# Remove null rows
#--Median Family Income has 748 null values which is 1.04%. This figure is insignificant compared to the importance of the
#feature towards prediction. So we deciide to drop null values than remove the column.
df=df.dropna()

In [5]:
#Find duplicate entries
print(f"Duplicate entries: {df.duplicated().sum()}")

Duplicate entries: 0


# Define Model

In [7]:
# Create features
X = df.drop(columns=['state_county',"LA1and10"]) 

# Create target
y = df["LA1and10"]

# Split model
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Logistic Regression

In [8]:
# Define model
model = LogisticRegression(solver='lbfgs', random_state=1)
# fit model

# Fit model
model.fit(X_train, y_train)
# make predictions on the entire training dataset
predictions = model.predict(X_test)

# Evaluate the model
y_pred = model.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.721


In [9]:
cm_logreg = confusion_matrix(y_test,y_pred)
cm_logreg

array([[567,   0],
       [219,   0]])

In [10]:
#Run Logistic Regression on scaled data
# Fit model
model.fit(X_train_scaled, y_train)
# make predictions on the entire training dataset
predictions_scaled = model.predict(X_test_scaled)

# Evaluate the model
y_pred_scaled = model.predict(X_test_scaled)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred_scaled):.3f}")

 Logistic regression model accuracy: 0.821


In [11]:
confusion_matrix(y_test,y_pred_scaled)

array([[529,  38],
       [103, 116]])

In [12]:
pd=pd.DataFrame({"Prediction": predictions, "Scaled Prediction":predictions_scaled, "Actual": y_test})
pd.head()

,Prediction,Scaled Prediction,Actual
1760,0,0,1
1735,0,0,1
1186,0,0,0
2803,0,0,1
3,0,0,0


### Undersampling

In [13]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [14]:
Counter(y)

Counter({1: 886, 0: 2256})

In [15]:
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 667, 1: 667})

In [16]:
# Retry Logistic Regression with undersampled data
model.fit(X_resampled, y_resampled)
y_pred_resamp = model.predict(X_test)
print(f" Resampled logistic regression model accuracy: {accuracy_score(y_test,y_pred_resamp):.3f}")
confusion_matrix(y_test, y_pred_resamp)

 Resampled logistic regression model accuracy: 0.721


array([[567,   0],
       [219,   0]])

### SVM

In [17]:
# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_svm = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred_svm):.3f}")

confusion_matrix(y_test, y_pred_svm)

 SVM model accuracy: 0.823


array([[533,  34],
       [105, 114]])

In [18]:
# Create the SVM model
svm_rbf = SVC()

# Train the model
svm_rbf.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_svmrbf = svm_rbf.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred_svmrbf):.3f}")

confusion_matrix(y_test, y_pred_svmrbf)

 SVM model accuracy: 0.823


array([[529,  38],
       [101, 118]])

### Random Forest

In [19]:
# Create Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [20]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data.
rf_y_pred = rf_model.predict(X_test_scaled)
# Evaluate the model
print(f"Random Forest model accuracy: {accuracy_score(y_test,rf_y_pred):.3f}")

Random Forest model accuracy: 0.822


In [21]:
confusion_matrix(y_test,rf_y_pred)

array([[511,  56],
       [ 84, 135]])

In [21]:
rf_model.feature_importances_


array([0.04668019, 0.90476039, 0.01152447, 0.00612747, 0.00951762,
       0.011283  , 0.01010687])

### Neural Network

In [22]:
import tensorflow as tf

In [23]:
# Define neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=14, activation="relu", input_dim=6))
#nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
74/74 [==============================] - 1s 1ms/step - loss: 0.6152 - accuracy: 0.7339
Epoch 2/50
74/74 [==============================] - 0s 1ms/step - loss: 0.5277 - accuracy: 0.7916
Epoch 3/50
74/74 [==============================] - 0s 1ms/step - loss: 0.4750 - accuracy: 0.8077
Epoch 4/50
74/74 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.8162
Epoch 5/50
74/74 [==============================] - 0s 1ms/step - loss: 0.4136 - accuracy: 0.8217
Epoch 6/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3959 - accuracy: 0.8251
Epoch 7/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3838 - accuracy: 0.8277
Epoch 8/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3755 - accuracy: 0.8281
Epoch 9/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3698 - accuracy: 0.8268
Epoch 10/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3660 - accuracy: 0.8256
Epoch 11/50
74/74 [

In [24]:
print(f'Logistic Regression: {accuracy_score(y_test,y_pred):.3f}')
print(f'Scaled Logistic Regression: {accuracy_score(y_test,y_pred_scaled):.3f}')
print(f'Undersampled Logistic Regression: {accuracy_score(y_test,y_pred_resamp):.3f}')
print(f'SVM Accuracy: {accuracy_score(y_test,y_pred_svm):.3f}')

Logistic Regression: 0.721
Scaled Logistic Regression: 0.821
Undersampled Logistic Regression: 0.721
SVM Accuracy: 0.823
